# Modellierung der Vorhersage der Probenhöhe


Dieses Notebook beschäftigt sich mit der Entwicklung von Vorhersagemodellen für die Zielgröße. Hierzu werden verschiedene Featuresets und Modelle verwendet, um die bestmögliche Vorhersagegenauigkeit zu erreichen.

## Einstellungen und Datenvorbereitung

- **Import relevanter Bibliotheken**
- **Metrik:** Die Leistung der Modelle wird anhand der `neg_mean_absolute_error`-Metriken bewertet.
- **Zielgröße (Target):** `Probenhoehe`
- **Irrelevante Spalten:** Diese Spalten werden aus dem Feature-Set entfernt, da sie für die Vorhersage nicht benötigt werden.

In [ ]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('..'))

from sklearn.model_selection import train_test_split, KFold
from IPython.display import display, HTML
from helpers.model_utils import (load_and_prepare_data, 
                                 display_top_models_for_target, 
                                 train_and_tune_models_regression, 
                                 compare_results_regression,  
                                 save_scores_to_csv, 
                                 analyze_model_performance, 
                                 plot_scores_and_percent_diff,
                                 direcetion_results,
                                 compare_smote_effects,
                                 filter_and_retrain_with_vif)
from helpers.model_pipelines import  (define_pipelines_Probenhoehe, 
                                      shap_analysis, 
                                      load_and_split_data,
                                      shap_analysis_single_model)


In [ ]:
# Einstellungen Notebook
target_name = "Probenhoehe"

# Metrik auswählen
metric = "neg_mean_absolute_error"  

# Zielgröße
target_column = 'Probenhoehe'  
irrelevant_columns = ['Material_con', 'Position_con', 'Ergebnis_con', 'richtig_verbaut', 'Zeit', 'VersuchID','umformzeit','synthetisch'
                      ]

# Einfluss der Verkippungssensoren
verkippungsfeatures = ["Verkippung_1", "Verkippung_2", "Verkippung_3", "Verkippung_4", 
                      "Verkippung_1_Min", "Verkippung_1_Max", "Verkippung_1_Mean", "Verkippung_1_Median", "Verkippung_1_Std", 
                      "Verkippung_2_Min","Verkippung_2_Max","Verkippung_2_Mean","Verkippung_2_Median","Verkippung_2_Std","Verkippung_3_Min",
                      "Verkippung_3_Max","Verkippung_3_Mean","Verkippung_3_Median","Verkippung_3_Std","Verkippung_4_Min","Verkippung_4_Max",
                      "Verkippung_4_Mean","Verkippung_4_Median","Verkippung_4_Std", "tilt_x_tiefster", "tilt_y_tiefster", "tilt_x_t0", "tilt_y_t0"]

# Mapping Pfad für Labels
label_mapping_path = "../mappings/label_mappings_binary.json"

# Verkippung berücksichtigen?
Verkippung = True

if not Verkippung:
    irrelevant_columns.extend(verkippungsfeatures)
    
# SHAP Analyse durchführen?
shap_on = False

plot_type = 'summary'  # Alternativ: "summary", "bar" oder "interaction" oder "violin"
plot_type = str(plot_type)


# Plots speichern?
save_plots = True
%config InlineBackend.figure_format = 'svg'

# mit SMOTE (synthethische Erweiterung der Trainingsdaten)?
smote_on = True

# Bautiel Temperatur berücksichtigen?
bauteil_temp = True
if not bauteil_temp:
    irrelevant_columns.append('Bauteil_Temp')

In [ ]:
# Pfade zu den Daten
test_filepath = f"../datasets/test_{target_column}.pkl"
if smote_on:
    # Pfade zu den SMOTE Trainingsdaten
    train_filepath = f"../datasets/train_{target_column}_smogn.pkl"
    test_filepath = f"../datasets/test_{target_column}_clean.pkl"
else: 
    save_plots = False
    train_filepath = f"../datasets/train_{target_column}_clean.pkl"
    test_filepath = f"../datasets/test_{target_column}_clean.pkl"

#test_filepath = f"../datasets/test_{target_column}_jittered.pkl"

# Lade und bereite Trainings- und Testdaten vor
X_train_original, y_train = load_and_prepare_data(train_filepath, target_column, irrelevant_columns)
X_test_original, y_test = load_and_prepare_data(test_filepath, target_column, irrelevant_columns)
    
# Überprüfen der Shapes
print(f"Trainingsdaten: X_train={X_train_original.shape}, y_train={y_train.shape}")
print(f"Testdaten: X_test={X_test_original.shape}, y_test={y_test.shape}")

# Zentrale Pipeline und Parameter
input_dim = X_train_original.shape[1]
target_pipeline = define_pipelines_Probenhoehe()
pipelines, param_grids = target_pipeline

# Cross-Validation Setup
# KFold für Regression
kf = KFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:
y_train = y_train.round(2)
y_test = y_test.round(2)

In [ ]:
y_train

# Process Features Dataset

In diesem Abschnitt wird das Datenset für die Prozessmerkmale vorbereitet und trainiert. Das Dataset wird angepasst, indem irrelevante Spalten entfernt und die relevanten Features extrahiert werden. Anschließend werden verschiedene Machine-Learning-Modelle trainiert, die darauf abzielen, die Zielgröße basierend auf den Prozessmerkmalen vorherzusagen.

In [ ]:
# Einstellungen Dataset
dataset_name = "process_features"
target_dir, feature_importance_path, balance_suffix, feature_importance_path_best, target_dir_best, tilt_suffix, temp_suffix = direcetion_results(dataset_name, target_name, smote_on, Verkippung, bauteil_temp)


# Auswahl der Process Features aus dem Dataset
columns_process_features = [
    "VersuchID",
    "Berührzeit",
    "Höhe_Wegmessung_Aufprall",
    "Umformzeit",
    "Tiefster_Punkt",
    "Energie_Aufprall",
    "Motorstrom_Durchschnitt",
    "Energie_ab_Durchschnitt",
    "Bauteil_Temp",
    "Werkzeug_Temp",
    "Material_con",
    "Position_con",
    "Ergebnis_con",
    "Probenhoehe",
]

# Sicherstellen, dass nur existierende Spalten verwendet werden
columns_to_keep_train = [col for col in columns_process_features if col in X_train_original.columns]

# X_train und X_test auf relevante Spalten beschränken
X_train = X_train_original[columns_to_keep_train]
X_test = X_test_original[columns_to_keep_train]

In [ ]:
X_train

## Training

In diesem Abschnitt wird das Training der Modelle durchgeführt. Die Modelle werden mit Hilfe von Cross-Validation optimiert, um eine robuste Bewertung der Modellleistung sicherzustellen. Hyperparameter-Tuning wird genutzt, um die besten Einstellungen für jedes Modell zu finden und die Generalisierungsfähigkeit auf Testdaten zu verbessern.

In [ ]:
# Modelltraining und Bewertung
input_dim = X_train.shape[1]
pipelines ,param_grids= define_pipelines_Probenhoehe(input_dim)
best_pipelines, results = train_and_tune_models_regression(
    pipelines, 
    param_grids, 
    X_train, 
    y_train, 
    kf, 
    X_test=X_test, 
    y_test=y_test
)
# Ergebnisse in DataFrame speichern
results_df_1 = pd.DataFrame.from_dict(results, orient='index').T
#results_df.drop(['best_params'], axis=0, inplace=True)

# Ergebnisse in tabellarischer Form anzeigen
display(HTML(results_df_1.to_html()))
#results_df_1.drop(['best_params'], axis=0, inplace=True)

# Ergebnisse speichern
save_scores_to_csv(
    results=results_df_1,  # Direkt das DataFrame übergeben
    output_dir=target_dir,    # Zielverzeichnis
    file_name=f"model_scores_{dataset_name}{balance_suffix}.csv",  # Dateiname
    Verkippung=Verkippung    # Flag für Verkippung
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_actual_vs_predicted(y_true, y_pred, model_name="Modell"):
    """
    Erstellt einen Scatter-Plot für tatsächliche vs. vorhergesagte Werte.
    
    Args:
        y_true (array-like): Tatsächliche Werte.
        y_pred (array-like): Vorhergesagte Werte.
        model_name (str): Name des Modells für den Titel.
    """
    plt.figure(figsize=(8, 6))
    
    # Streudiagramm
    plt.scatter(y_pred, y_true, color='red', alpha=0.6, label="Datenpunkte")
    
    # Diagonale Linie (perfekte Vorhersage)
    min_val = min(min(y_true), min(y_pred))
    max_val = max(max(y_true), max(y_pred))
    plt.plot([min_val, max_val], [min_val, max_val], 'k--', label="Perfekte Vorhersage")
    
    # Achsenbeschriftung
    plt.xlabel("Vorhergesagte Probenhöhe (mm)")
    plt.ylabel("Tatsächliche Probenhöhe (mm)")
    plt.title(f"Tatsächliche vs. vorhergesagte Probenhöhe ({model_name})")
    #plt.legend()
    plt.grid(True)
    
    plt.show()

In [ ]:
# Schritt 2: Vorhersagen für ein Modell generieren (z. B. XGBoost)

for model_name in best_pipelines:
    model = best_pipelines[model_name]
    y_pred = model.predict(X_test)

    # Schritt 3: Visualisierung
    plot_actual_vs_predicted(y_test, y_pred, model_name=model_name)
else:
    print(f"Fehler: Modell '{model_name}' wurde nicht gefunden.")

## Overfitting Test

In diesem Abschnitt wird die Überprüfung von Overfitting durch die Analyse von Learning Curves und Validation Curves durchgeführt, um den Bias-Variance-Tradeoff besser zu verstehen. Die `analyze_model_performance` Funktion aus der `model_pipelines.py` führt diese Analysen durch und erstellt entsprechende Plots. Es werden sowohl die Trainingsscores als auch die Testscores über verschiedene Größen des Trainingssets und Hyperparameterbereiche betrachtet, um die Modellstabilität und die Generalisierungsfähigkeit zu bewerten. Dies hilft dabei, die optimale Komplexität der Modelle zu bestimmen und sicherzustellen, dass sie gut auf unbekannte Daten generalisieren.

In [ ]:
# Overfitting überprüfen 
# Analyse von Learning Curves für Bias-Variance-Tradeoff
print(f"\nLearning Curve Analyse für {target_name}:")

# Analyse der Modellleistung
analyze_model_performance(
    pipelines=pipelines, 
    param_grids=param_grids, 
    X=X_train, 
    y=y_train, 
    scoring=metric, 
    cv=kf,
    save_plots=save_plots,
    output_dir= target_dir,
    Verkippung=Verkippung
)


In [ ]:
# Alternative Validierungsverfahren
#print("\nAlternative Validierungsverfahren:")
#for name, pipeline in best_pipelines.items():
#    print(f"Alternative Validierungsverfahren für {name}...")
#    test_alternative_validation(pipeline, X_train, y_train)

## SHAP-Analyse

In diesem Abschnitt führen wir eine SHAP-Analyse durch, um die Einflüsse der einzelnen Features auf die Vorhersagen der besten Modelle zu verstehen. Die Ergebnisse dieser Analyse helfen uns, die Modellentscheidungen transparenter zu machen. Anschließend laden wir Diagramme, die die besten drei Modelle basierend auf unterschiedlichen Metriken darstellen. Diese Visualisierungen unterstützen die Bewertung der Modellperformance und die Identifizierung von Schlüsselfeatures, die die Zielgröße beeinflussen.

In [ ]:
# SHAP-Analyse
if shap_on:
    shap_analysis(
        best_pipelines=best_pipelines,
        X_train=X_train,
        target_name=target_name,
        dataset_name=dataset_name,
        output_dir=target_dir,
        plot_type=plot_type,  # Alternativ: "bar" oder "interaction" oder "violin"
        save_plots=save_plots,
        verkippung=Verkippung
    ) 

In [ ]:
# Diagramme aus der Zielgröße laden
display_top_models_for_target(
    target_dir,
    results_df_1,
    metric= 'Test MSE',
    top_n=3,  # Nur die besten 3 Modelle anzeigen
)

# Aggregierte Daten

In [ ]:
# Einstellungen
dataset_name = "aggregated_features"
target_dir, feature_importance_path, balance_suffix, feature_importance_path_best, target_dir_best, tilt_suffix, temp_suffix = direcetion_results(dataset_name, target_name, smote_on, Verkippung, bauteil_temp)



# Auswahl der Process Features aus dem Dataset
columns_aggregated_features = [
    "VersuchID",
    "Berührzeit",
    "Höhe_Wegmessung_Aufprall",
    "Umformzeit",
    "Tiefster_Punkt",
    "Energie_Aufprall",
    "Motorstrom_Durchschnitt",
    "Energie_ab_Durchschnitt",
    "Bauteil_Temp",
    "Werkzeug_Temp",
    "Material_con",
    "Position_con",
    "Ergebnis_con",
    "Probenhoehe",
    "richtig_verbaut",
    "Wegmessung_Min",
    "Wegmessung_Max",
    "Wegmessung_Mean",
    "Wegmessung_Median",
    "Wegmessung_Std",
    "Verkippung_1_Min",
    "Verkippung_1_Max",
    "Verkippung_1_Mean",
    "Verkippung_1_Median",
    "Verkippung_1_Std",
    "Verkippung_2_Min",
    "Verkippung_2_Max",
    "Verkippung_2_Mean",
    "Verkippung_2_Median",
    "Verkippung_2_Std",
    "Verkippung_3_Min",
    "Verkippung_3_Max",
    "Verkippung_3_Mean",
    "Verkippung_3_Median",
    "Verkippung_3_Std",
    "Verkippung_4_Min",
    "Verkippung_4_Max",
    "Verkippung_4_Mean",
    "Verkippung_4_Median",
    "Verkippung_4_Std",
    "Stoesselhub_Min",
    "Stoesselhub_Max",
    "Stoesselhub_Mean",
    "Stoesselhub_Median",
    "Stoesselhub_Std",
    "Geschwindigkeit_Ges_Min",
    "Geschwindigkeit_Ges_Max",
    "Geschwindigkeit_Ges_Mean",
    "Geschwindigkeit_Ges_Median",
    "Geschwindigkeit_Ges_Std",
    "Presskraft_dyn_Min",
    "Presskraft_dyn_Max",
    "Presskraft_dyn_Mean",
    "Presskraft_dyn_Median",
    "Presskraft_dyn_Std",
    "Motorstrom_Min",
    "Motorstrom_Max",
    "Motorstrom_Mean",
    "Motorstrom_Median",
    "Motorstrom_Std",
]

# Sicherstellen, dass nur existierende Spalten verwendet werden
columns_to_keep_train = [col for col in columns_aggregated_features if col in X_train_original.columns]

# X_train und X_test auf relevante Spalten beschränken
X_train = X_train_original[columns_to_keep_train]
X_test = X_test_original[columns_to_keep_train]

## Training

In [ ]:
# Modelltraining und Bewertung
input_dim = X_train.shape[1]
pipelines ,param_grids= define_pipelines_Probenhoehe(input_dim)
best_pipelines, results = train_and_tune_models_regression(
    pipelines, 
    param_grids, 
    X_train, 
    y_train, 
    kf, 
    X_test=X_test, 
    y_test=y_test
)

# Ergebnisse in DataFrame speichern
results_df_2 = pd.DataFrame.from_dict(results, orient='index').T
#results_df.drop(['best_params'], axis=0, inplace=True)

# Ergebnisse in tabellarischer Form anzeigen
display(HTML(results_df_2.to_html()))

save_scores_to_csv(
    results=results_df_2,  # DataFrame mit den Scores
    output_dir=target_dir,
    file_name=f"model_scores_{dataset_name}{balance_suffix}.csv",  # Gemeinsamer Basisname
    Verkippung=Verkippung  # Fügt den '_no_tilt'-Suffix hinzu, falls nötig
)

In [ ]:
# Schritt 2: Vorhersagen für ein Modell generieren (z. B. XGBoost)
for model_name in best_pipelines:
    model = best_pipelines[model_name]
    y_pred = model.predict(X_test)

    # Schritt 3: Visualisierung
    plot_actual_vs_predicted(y_test, y_pred, model_name=model_name)
else:
    print(f"Fehler: Modell '{model_name}' wurde nicht gefunden.")

## Overfitting Test

In [ ]:
# Overfitting überprüfen 
# Analyse von Learning Curves für Bias-Variance-Tradeoff
print(f"\nLearning Curve Analyse für {target_name}:")

# Analyse der Modellleistung
analyze_model_performance(
    pipelines=pipelines, 
    param_grids=param_grids, 
    X=X_train, 
    y=y_train, 
    scoring=metric, 
    cv=kf,
    save_plots=save_plots,
    output_dir= target_dir,
    Verkippung=Verkippung
)


## SHAP-Analyse

In [ ]:
# SHAP-Analyse
if shap_on:
    shap_analysis(
        best_pipelines=best_pipelines,
        X_train=X_train,
        target_name=target_name,
        dataset_name=dataset_name,
        output_dir=target_dir,
        plot_type=plot_type,  # Alternativ: "bar" oder "interaction" oder "violin"
        save_plots=save_plots,
        verkippung=Verkippung
    ) 

In [ ]:
# Alle Diagramme aus der Zielgröße "Prozessqualität" laden
display_top_models_for_target(
    target_dir,
    results_df_2,
    metric = 'Test MSE',
    top_n=3  # Nur die besten 3 Modelle anzeigen
)

# New Features Dataset

In [ ]:
# Einstellungen
dataset_name = "new_features"
target_dir, feature_importance_path, balance_suffix, feature_importance_path_best, target_dir_best, tilt_suffix, temp_suffix = direcetion_results(dataset_name, target_name, smote_on, Verkippung, bauteil_temp)

# X_train und X_test auf relevante Spalten beschränken
X_train = X_train_original
X_test = X_test_original

In [ ]:
# X_train und X_test auf relevante Spalten beschränken
X_train = X_train_original.drop(columns='synthetisch', errors='ignore')


## Training

In [ ]:
# Modelltraining und Bewertung
input_dim = X_train.shape[1]
pipelines ,param_grids= define_pipelines_Probenhoehe(input_dim)
best_pipelines, results = train_and_tune_models_regression(
    pipelines, 
    param_grids, 
    X_train, 
    y_train, 
    kf, 
    X_test=X_test, 
    y_test=y_test
)

# Ergebnisse in DataFrame speichern
results_df_3 = pd.DataFrame.from_dict(results, orient='index').T
#results_df.drop(['best_params'], axis=0, inplace=True)

# Ergebnisse in tabellarischer Form anzeigen
display(HTML(results_df_3.to_html()))

save_scores_to_csv(
    results=results_df_3,  # DataFrame mit den Scores
    output_dir=target_dir,
    file_name=f"model_scores_{dataset_name}{balance_suffix}.csv",  # Gemeinsamer Basisname
    Verkippung=Verkippung  # Fügt den '_no_tilt'-Suffix hinzu, falls nötig
)

In [ ]:
# Schritt 2: Vorhersagen für ein Modell generieren (z. B. XGBoost)
for model_name in best_pipelines:
    model = best_pipelines[model_name]
    y_pred = model.predict(X_test)

    # Schritt 3: Visualisierung
    plot_actual_vs_predicted(y_test, y_pred, model_name=model_name)
else:
    print(f"Fehler: Modell '{model_name}' wurde nicht gefunden.")

## Overfitting Test

In [ ]:
# Overfitting überprüfen 
# Analyse von Learning Curves für Bias-Variance-Tradeoff
print(f"\nLearning Curve Analyse für {target_name}:")

# Analyse der Modellleistung
analyze_model_performance(
    pipelines=pipelines, 
    param_grids=param_grids, 
    X=X_train, 
    y=y_train, 
    scoring=metric, 
    cv=kf,
    save_plots=save_plots,
    output_dir= target_dir,
    Verkippung=Verkippung
)


## SHAP-Analyse

In [ ]:
# SHAP-Analyse
if shap_on:
    shap_analysis(
        best_pipelines=best_pipelines,
        X_train=X_train,
        target_name=target_name,
        dataset_name=dataset_name,
        output_dir=target_dir,
        plot_type=plot_type,  # Alternativ: "bar" oder "interaction" oder "violin"
        save_plots=save_plots,
        verkippung=Verkippung
    ) 

# Vergleich der Results 


In [ ]:
results_dfs = {
    "Process Features": results_df_1,
    "Aggregated Features": results_df_2,
    "New Features": results_df_3
}

# Vergleich der Ergebnisse über die Datasets hinweg
compare_metric = "Test MSE"
comparison_table, best_combination = compare_results_regression(results_dfs, metric=compare_metric)

# Vergleichstabelle in HTML-Format konvertieren und anzeigen
print(f"Vergleich der {compare_metric}-Scores für die unterschiedlichen Datasets und Modelle:")
display(HTML(comparison_table.to_html()))

# Beste Kombination ausgeben
print(f"\nBeste Kombination:")
print(f"Dataset: {best_combination[0]}, Modell: {best_combination[1]}, Score: {best_combination[2]:.4f}")

save_scores_to_csv(
    results=comparison_table,
    output_dir=target_dir,
    file_name=f"compare_model_scores_{target_name}_{compare_metric}.csv",
    Verkippung=Verkippung
)

In [ ]:
# def plot_f1_scores_and_percent_diff_MSE(results_dfs, target_name, output_path=None):
#     """
#     Erstellt ein Balkendiagramm für den MSE über verschiedene Feature-Sätze und ergänzt
#     eine Liniengrafik für die prozentuale Abweichung (percent_diff). Optional wird das Diagramm gespeichert.
    
#     Args:
#         results_dfs (dict): Dictionary mit den DataFrames für verschiedene Feature-Sätze.
#         target_name (str): Name des Zielwerts (Target), der im Diagrammtitel verwendet wird.
#         output_path (str): Pfad, unter dem das Diagramm gespeichert werden soll (optional).
#     """

#     # Modelle aus einem der DataFrames extrahieren
#     models = results_dfs["Process Features"].columns

#     # MSE für verschiedene Feature-Sätze abrufen
#     MSE_process = results_dfs["Process Features"].loc["Test MSE"].values
#     MSE_aggregated = results_dfs["Aggregated Features"].loc["Test MSE"].values
#     MSE_new = results_dfs["New Features"].loc["Test MSE"].values

#     # Prozentuale Abweichung (percent_diff_MSE) abrufen
#     percent_diff_process = results_dfs["Process Features"].loc["percent_diff_MSE"].values
#     percent_diff_aggregated = results_dfs["Aggregated Features"].loc["percent_diff_MSE"].values
#     percent_diff_new = results_dfs["New Features"].loc["percent_diff_MSE"].values

#     # Breite der Balken und Positionen auf der X-Achse
#     bar_width = 0.2
#     x = np.arange(len(models))

#     # Erstellen des Balkendiagramms für MSE
#     fig, ax1 = plt.subplots(figsize=(12, 6))

#     ax1.bar(x - bar_width, MSE_process, width=bar_width, label="Prozess Features", color='firebrick')
#     ax1.bar(x, MSE_aggregated, width=bar_width, label="Aggregierte Features", color='goldenrod')
#     ax1.bar(x + bar_width, MSE_new, width=bar_width, label="Neue Features", color='forestgreen')
    
#     # Achsenbeschriftung und Titel
#     ax1.set_xlabel("Modelle")
#     ax1.set_ylabel("MSE", color='black')
#     ax1.set_title("Vergleich der Modellleistung für die {} mit unterschiedlichen Feature-Datensätzen".format(target_name))
#     ax1.set_xticks(x)
#     ax1.set_xticklabels(models, rotation=45)
#     ax1.legend(loc="upper left")

#     # Skalierung der linken Y-Achse anpassen
#     ax1.set_ylim(0, 1)  # MSE zwischen 0 und 1

#     # Zweite Y-Achse für percent_diff
#     ax2 = ax1.twinx()
#     ax2.plot(x, percent_diff_process, marker='o', linestyle='-', color='maroon', label="Prozentuale Differenz (Prozess)")
#     ax2.plot(x, percent_diff_aggregated, marker='s', linestyle='-', color='darkgoldenrod', label="Prozentuale Differenz (Aggregiert)")
#     ax2.plot(x, percent_diff_new, marker='^', linestyle='-', color='darkgreen', label="Prozentuale Differenz (Neu)")

#     # Skalierung der rechten Y-Achse anpassen
#     max_percent_diff = max(
#         np.max(percent_diff_process),
#         np.max(percent_diff_aggregated),
#         np.max(percent_diff_new)
#     )
#     ax2.set_ylim(0, max_percent_diff * 1.2)  # Dynamische Skalierung mit etwas Puffer

#     ax2.set_ylabel("%-Abweichung zwischen Train- & Testdatensatz", color='black')
#     ax2.legend(loc="upper right")

#     # Diagramm speichern, falls ein Pfad angegeben ist
#     if output_path:
#         # Verzeichnis erstellen, falls es nicht existiert
#         os.makedirs(os.path.dirname(output_path), exist_ok=True)
#         plt.savefig(output_path, dpi=300, bbox_inches='tight')  # Speichert das Diagramm mit hoher Auflösung

#     # Diagramm anzeigen
#     plt.tight_layout()
#     plt.show()


In [ ]:
output_path = f"../results/{target_name}/{balance_suffix}{tilt_suffix}{temp_suffix}/compare_model_scores_{target_name}_{compare_metric}{balance_suffix}{tilt_suffix}{temp_suffix}.svg"

# Diagramm erstellen
# RMSE macht mehr Sinn prozentual zu vergleichen als der MSE, da dieser in Original Einheiten ist
plot_scores_and_percent_diff(results_dfs, target_name=target_name, output_path=output_path, y_lim=(0, 0.4), metric="Test MSE")

# Reduktion der Features & Performance des Best Models
- Das beste Modell wird nach der Feature Reduktion mit Hilfe der SHAP Values im Zielverzeichis gespeichert.

In [ ]:
# Modell und Feature Datensatz auswählen
model_name = "XGBoost"
pipeline = best_pipelines[model_name]
dataset_name = "new_features"

target_dir, feature_importance_path, balance_suffix, feature_importance_path_best, target_dir_best, tilt_suffix, temp_suffix = direcetion_results(dataset_name, target_name, smote_on, Verkippung, bauteil_temp, model_name=model_name)

# Verzeichnis erstellen, falls es nicht existiert
os.makedirs(target_dir, exist_ok=True)

In [ ]:
feature_importance_path_best

In [ ]:
# SHAP-Analyse für ein einzelnes Modell
shap_analysis_single_model(
    model_name=model_name,
    pipeline=pipeline,
    X_train=X_train,
    target_name=target_name,
    dataset_name=dataset_name,
    output_dir=target_dir_best,
    plot_type=plot_type,
    save_plots=save_plots,
    verkippung=Verkippung
)

In [ ]:
# **AUFRUF DER FUNKTION**
best_pipelines, retrain_results = filter_and_retrain_with_vif(
    model_name=model_name,
    pipeline=pipeline,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    feature_importance_path=feature_importance_path_best,
    param_grids=param_grids,
    cv=kf,
    scoring=metric,
    pareto_split=0.8,  
    correlation_threshold=0.9,  
    vif_threshold=75.0,  
    output_dir=target_dir_best,
    target_name=target_name,
    dataset_name=dataset_name,
    verkippung=Verkippung,
    is_regression=True,
    label_mapping_path=label_mapping_path,    # Neuer Parameter für Confusion-Matrix
    target_column=target_column          # Neuer Parameter für Confusion-Matri  
)

# Visualisierungen


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_actual_vs_predicted(y_true, y_pred, model_name="Modell"):
    """
    Erstellt einen Scatter-Plot für tatsächliche vs. vorhergesagte Werte.
    
    Args:
        y_true (array-like): Tatsächliche Werte.
        y_pred (array-like): Vorhergesagte Werte.
        model_name (str): Name des Modells für den Titel.
    """
    plt.figure(figsize=(8, 6))
    
    # Streudiagramm
    plt.scatter(y_pred, y_true, color='red', alpha=0.6, label="Datenpunkte")
    
    # Diagonale Linie (perfekte Vorhersage)
    min_val = min(min(y_true), min(y_pred))
    max_val = max(max(y_true), max(y_pred))
    plt.plot([min_val, max_val], [min_val, max_val], 'k--', label="Perfekte Vorhersage")
    
    # Achsenbeschriftung
    plt.xlabel("Vorhergesagte Probenhöhe (mm)")
    plt.ylabel("Tatsächliche Probenhöhe (mm)")
    plt.title(f"Tatsächliche vs. vorhergesagte Probenhöhe ({model_name})")
    plt.legend()
    plt.grid(True)
    
    plt.show()

## SMOTE für Prozess Features


In [ ]:
smote_path = f"../results/{target_name}/{tilt_suffix}{temp_suffix}/process_features/model_scores_process_features.csv"
no_smote_path = f"../results/{target_name}/noSMOTE/process_features/model_scores_process_featuresnoSMOTE.csv"
output_path = f"../results/{target_name}/compare_SMOTE_process_features_{target_name}_{compare_metric}.svg"
%config InlineBackend.figure_format = 'svg'
df_comp = compare_smote_effects(no_smote_path, smote_path, metric="Test MSE", title=f"Vergleich der Fehlermetrik für das Prozess Feature-Set: No SMOGN vs. SMOGN", save_path=output_path)
